https://z-uo.medium.com/lenguage-translations-with-python-5dd745d1067f

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/Users/mstudio/miniforge3/envs/hemingway/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
it_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

/Users/mstudio/miniforge3/envs/hemingway/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
it_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

In [12]:
text = "In my younger and more vulnerable years my father gave me some advice that I’ve been turning over in my mind ever since."
#내가 아직 어리고 지금보다 훨씬 더 쉽게 남의 말에 곧잘 화를 내던 시절 아버지께서는 나에게 한 가지 충고를 해 주셨는데, 그 후로 나는 그 충고를 마음속으로 항상 되새기곤 했다.
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text)

['Un remained\xad endWhere\xadshiny remained\xad Stepweight\xad values US." 슈퍼 전시품은 대담하게 남아 있습니다.']


In my younger and more vulnerable years my father gave me some advice that I’ve been turning over in my mind ever since.

https://github.com/rainmaker712/nlp_ryan/blob/master/Chatbot/Slack_Bot/data/%EC%9C%84%EB%8C%80%ED%95%9C%EA%B0%9C%EC%B8%A0%EB%B9%84.txt

http://davincimap.co.kr/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR001670&Lang=%EC%98%81%EB%AC%B8&Page=1


He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 

In [ ]:
#문학동네
text = "그는 멕시코 만류에서 조그만 돛단배로 혼자 고기잡이를 하는 노인이었다. 팔십사 일 동안 그는 바다에 나가서 고기를 한 마리도 못잡았다."
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text)

['He was an old man who fished alone with a small sailing boat in the Gulf of Mexico, and for eighty-four days he went out to sea and could not catch a fish.']


In [ ]:
#민음사
text = "그는 멕시코 만류에서 조각배를 타고 홀로 고기잡이 하는 노인이었다. 여든 날 하고도 나흘이 지나도록 고기 한 마리 낚지 못했다."
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text)

['He was an old man riding on a rock boat in the Gulf of Mexico and fishing alone, and he could not catch a fish four days after eighty days.']


In [ ]:
#열린책들
text = "그는 걸프 해류에서 조각배를 타고서 혼자 낚시하는 노인이었고, 고기를 단 한 마리도 잡지 못한 날이 이제 84일이었다."
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text)

['He was an old man riding on a rock boat in the Gulf Current, fishing alone, and it was now 84 days when not a single fish could be caught.']


In [ ]:
#시공사
text = "그는 멕시코 만류가 흐르는 지역에서 작은 배를 타고 혼자 고기잡이를 하는 노인이었다 오늘까지 84일 동안 그는 고기를 한 마리도 낚지 못한 채 시간을 보냈다."
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text)

['He was an old man who fished alone in a small boat in an area of the Gulf of Mexico until today, 84 days without catching even one fish.']


In [ ]:
#다상
text="그는 멕시코 만류에서 조각배를 타고 홀로 고기잡이를 해서 살아가는 노인이었는데, 고기를 한 마리도 잡지 못하고 벌써 84일이 지났다."
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text) 

['He was an old man who was living alone on a rock boat in the Gulf of Mexico, but he had not caught any meat, and it had already been 84 days.']


In [ ]:
#삼지사
text="그는 멕시코 만류에 조각배를 띄우고 홀로 고기 잡는 노인이었는데, 고기 한 마리 낚지 못하는 날이 벌써 84일이나 계속되고 있었다."
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text) 

['He was an old man who had laid out a rock boat in the Gulf of Mexico and was a lone fisherman, and the day was already 84 days without a fish.']


In [ ]:
#북로드
text="그는 홀로 조각배를 타고 멕시코만류에 나가 고기를 잡는 노인이었다. 노인은 지난 84일 동안 고기를 한 마리도 잡지 못했다."
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text)

['He was an old man who was alone in a carton, going out to the Gulf of Mexico to catch fish, and he had not caught a single fish in the past 84 days.']


In [ ]:
#계몽사
text="그는 멕시코 만류가 흐르는 바다에서 홀로 작은 배를 몰며 고기잡이를 하는 노인이었다. 그 노인은 여든하고 네번째 날이 지나도록 작은 고기 한마리조차 낚지 못했다."
encoded_input = it_en_tokenizer(text, return_tensors="pt")
output = it_en_model.generate(**encoded_input)
out_text = it_en_tokenizer.batch_decode(output,      
                                       skip_special_tokens=True)
print(out_text) 

['He was an old man who was alone in fishing in the waters of the Gulf of Mexico, and he could not catch even a small fish until the fourth day of the year.']


In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 3.8 MB 32.7 MB/s 
     |████████████████████████████████| 1.2 MB 35.2 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
     |████████████████████████████████| 6.5 MB 44.6 MB/s 
     |████████████████████████████████| 895 kB 34.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=7b168551c9b6150db1f2728a3cc932feedea799db1cec701f917feead72a145c
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.']

sentences2 = ['He was an old man who fished alone with a small sailing boat in the Gulf of Mexico, and for eighty-four days he went out to sea and could not catch a fish.', #문학동네
              'He was an old man riding on a rock boat in the Gulf of Mexico and fishing alone, and he could not catch a fish four days after eighty days.', #민음사
              'He was an old man riding on a rock boat in the Gulf Current, fishing alone, and it was now 84 days when not a single fish could be caught.', #열린책들
              'He was an old man who fished alone in a small boat in an area of the Gulf of Mexico until today, 84 days without catching even one fish.', #시공사
              'He was an old man who was living alone on a rock boat in the Gulf of Mexico, but he had not caught any meat, and it had already been 84 days.', #다상
              'He was an old man who had laid out a rock boat in the Gulf of Mexico and was a lone fisherman, and the day was already 84 days without a fish.', #삼지사
              'He was an old man who was alone in a carton, going out to the Gulf of Mexico to catch fish, and he had not caught a single fish in the past 84 days.', #북로드
              'He was an old man who was alone in fishing in the waters of the Gulf of Mexico, and he could not catch even a small fish until the fourth day of the year.', #계몽사
              ]

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \n{} \nScore: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man who fished alone with a small sailing boat in the Gulf of Mexico, and for eighty-four days he went out to sea and could not catch a fish. 
Score: 0.8891
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man riding on a rock boat in the Gulf of Mexico and fishing alone, and he could not catch a fish four days after eighty days. 
Score: 0.8511
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man riding on a rock boat in the Gulf Current, fishing alone, and it was now 84 days when not a single fish could be caught. 
Score: 0.8848
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old ma

https://github.com/monologg/kakaotrans

In [1]:
from kakaotrans import Translator
translator=Translator()
from tqdm import tqdm
import time

In [2]:
f=open('rest.txt','r',encoding='utf-8')
translated=[]
for i in tqdm(f.readlines()):
    try:
        trans=translator.translate(i)
        translated.append(trans)
        time.sleep(0.5)
    except TypeError:
        translated.append(i)
        time.sleep(0.5)
        continue
f.close()

100%|██████████| 127/127 [04:23<00:00,  2.07s/it]


In [3]:
with open('kakao_E2K_rest.txt', 'w') as f:
    for line in translated:
        f.write(line)
    f.close()

In [27]:
translated[1124]

'어쨌든 바람은 우리의 친구라고 그는 생각했다. 그리고 그는 가끔 덧붙였다. 그리고 우리의 친구들과 적들과 함께 있는 거대한 바다. 침대는 내 친구야. 그냥 침대라고, 그는 생각했다. 침대는 좋은 것이 될 것이다. 그는 네가 맞으면 쉽다고 생각했다. 나는 그것이 얼마나 쉬운지 결코 알지 못했다. 그리고 무엇이 너를 이겼는지, 그는 생각했다.'

In [ ]:
translator.translate('그는 멕시코 만류에서 조그만 돛단배로 혼자 고기잡이를 하는 노인이었다. 팔십사 일 동안 그는 바다에 나가서 고기를 한 마리도 못잡았다.', src='kr', tgt='en')

'He was an old man who was fishing alone in a small sailboat in the Gulf of Mexico, and for eighty-four days he had gone out to sea and not caught a single fish.'

In [ ]:
translator.translate('그는 멕시코 만류에서 조각배를 타고 홀로 고기잡이 하는 노인이었다. 여든 날 하고도 나흘이 지나도록 고기 한 마리 낚지 못했다.', src='kr', tgt='en')

'He was an old man who was fishing alone in a carving boat in the Gulf Stream of Mexico, and he had not fished a fish for four days after eighty days.'

In [ ]:
translator.translate('그는 걸프 해류에서 조각배를 타고서 혼자 낚시하는 노인이었고, 고기를 단 한 마리도 잡지 못한 날이 이제 84일이었다.', src='kr', tgt='en')

'He was an old man fishing alone in a carving boat in the Gulf Current, and the day he could not catch a single meat was now 84 days.'

In [ ]:
translator.translate('그는 멕시코 만류가 흐르는 지역에서 작은 배를 타고 혼자 고기잡이를 하는 노인이었다 오늘까지 84일 동안 그는 고기를 한 마리도 낚지 못한 채 시간을 보냈다.', src='kr', tgt='en')

'He was an old man who fished alone in a small boat in the Gulf Stream of Mexico. He spent 84 days without fishing a fish.'

In [ ]:
translator.translate('그는 멕시코 만류에서 조각배를 타고 홀로 고기잡이를 해서 살아가는 노인이었는데, 고기를 한 마리도 잡지 못하고 벌써 84일이 지났다.', src='kr', tgt='en')

'He was an old man who lived alone in a carving boat in the Gulf of Mexico, and he had not caught a meat and had already passed 84 days.'

In [ ]:
translator.translate('그는 멕시코 만류에 조각배를 띄우고 홀로 고기 잡는 노인이었는데, 고기 한 마리 낚지 못하는 날이 벌써 84일이나 계속되고 있었다.', src='kr', tgt='en')

'He was an old man who floated a carving boat in the Gulf of Mexico and caught a meat alone, and the day that he could not catch a meat had already been 84 days.'

In [ ]:
translator.translate('그는 홀로 조각배를 타고 멕시코만류에 나가 고기를 잡는 노인이었다 노인은 지난 84일 동안 고기를 한 마리도 잡지 못했다.', src='kr', tgt='en')

'He was an old man who went out to the Gulf of Mexico alone in a sculpture boat and caught meat. The old man did not catch any meat for the past 84 days.'

In [ ]:
translator.translate('그는 멕시코 만류가 흐르는 바다에서 홀로 작은 배를 몰며 고기잡이를 하는 노인이었다. 그 노인은 여든하고 네번째 날이 지나도록 작은 고기 한마리조차 낚지 못했다.', src='kr', tgt='en')

'He was an old man who was fishing alone in a small boat in the waters of the Gulf of Mexico, and he could not catch a small fish until the eighth and fourth days passed.'

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.',
              'He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish.']

sentences2 = ['He was an old man who was fishing alone in a small sailboat in the Gulf of Mexico, and for eighty-four days he had gone out to sea and not caught a single fish.', #문학동네
              'He was an old man who was fishing alone in a carving boat in the Gulf Stream of Mexico, and he had not fished a fish for four days after eighty days.', #민음사
              'He was an old man fishing alone in a carving boat in the Gulf Current, and the day he could not catch a single meat was now 84 days.', #열린책들
              'He was an old man who fished alone in a small boat in the Gulf Stream of Mexico. He spent 84 days without fishing a fish.', #시공사
              'He was an old man who lived alone in a carving boat in the Gulf of Mexico, and he had not caught a meat and had already passed 84 days.', #다상
              'He was an old man who floated a carving boat in the Gulf of Mexico and caught a meat alone, and the day that he could not catch a meat had already been 84 days.', #삼지사
              'He was an old man who went out to the Gulf of Mexico alone in a sculpture boat and caught meat. The old man did not catch any meat for the past 84 days.', #북로드
              'He was an old man who was fishing alone in a small boat in the waters of the Gulf of Mexico, and he could not catch a small fish until the eighth and fourth days passed.', #계몽사
              ]

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \n{} \nScore: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man who was fishing alone in a small sailboat in the Gulf of Mexico, and for eighty-four days he had gone out to sea and not caught a single fish. 
Score: 0.8781
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man who was fishing alone in a carving boat in the Gulf Stream of Mexico, and he had not fished a fish for four days after eighty days. 
Score: 0.8912
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was an old man fishing alone in a carving boat in the Gulf Current, and the day he could not catch a single meat was now 84 days. 
Score: 0.8553
He was an old man who fished alone in a skiff in the Gulf Stream and he had gone eighty-four days now without taking a fish. 
He was 

In [ ]:
from collections import Counter
original=[
          'He','was','an','old','man','who','fished','alone','in','a','skiff','in','the', 'Gulf','Stream','and','he','had','gone','eighty-four','days','now','without','taking','a','fish'
]
translation=[
          'He','was','an','old','man','who','was','fishing','alone','in','a','small','boat','in','the','waters','of','the','Gulf','of','Mexico','and', 'he', 'could', 'not', 'catch', 'a', 'small', 'fish', 'until', 'the', 'eighth', 'and', 'fourth', 'days', 'passed' 

]
list((Counter(original)-Counter(translation)).elements())

['fished',
 'skiff',
 'Stream',
 'had',
 'gone',
 'eighty-four',
 'now',
 'without',
 'taking']